In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
def read_POSCAR(file_name="POSCAR"):
	'''Read structure files of VASP program.
    [Input] : file's name. Default="POSCAR"
    [Output] : system_name, lattice, cell_vec, species, natoms, coord_type, coord, Selective, selective_tags
	|-> system_name : name of system. @str
    |-> lattice : scale @float
    |-> cell_vec: [x1, y1, z1], [x2, y2, z2], [x3, y3, z3]] @np.array(dtype='d')
    |-> species: [atomt1, atom2, ...] @list(str)
    |-> natoms: [number_of_atom1, number_of_atom2, ...] @list(int)
	|-> coord_type: "DIRECT" / "CARTESIAN" @str
	|-> coord : coordination of each atoms @np.array(dtype='d')
    |-> Selective : True / False @bool
    |-> selective_tags : [[T/F, T/F, T/F], [T/F, T/F, T/F], ...] @np.array(dtype=str)
    '''
	ip=open(file_name, 'r')
	system_name=ip.readline().strip()
	lattice=float(ip.readline().strip())
	#cell vector
	cell_vec=[]
	for i in range(3):
	    cell_vec.append(ip.readline().split())
	cell_vec=np.array(cell_vec,dtype="d")

	# atomic information (species, number)
	species=ip.readline().split()
	nspecies=len(species)
	natoms=list(map(lambda x:int(x),ip.readline().split()))
	tot_natoms=sum(natoms)

	# Selective Dynamics & Coord_Type
	what1=ip.readline().strip()
	if what1.upper().startswith("S"):
		Selective=True
		what2=ip.readline().strip()
		if what2.upper().startswith("D"):
			Cartesian=False
		elif what2.upper().startswith("F"):
			Cartesian=False
		else:
			Cartesian=True
	else:
		Selective=False
		if what1.upper().startswith("D"):
			Cartesian=False
		elif what1.upper().startswith("F"):
			Cartesian=False
		else:
			Cartesian=True
	if Cartesian:
		coord_type="CARTESIAN"
	else:
		coord_type="DIRECT"

	# Coordination
	coord=[]
	selective_tags=[]
	if not(Selective):
		for i in range(tot_natoms):
			coord.append(ip.readline().split())
		try:
			coord=np.array(coord,dtype="d")
		except:
			raise IOError("POSCAR file is weird. In [%s] file, Maybe number of coordinations are not matched with total number of atoms"%(file_name))
	else:
		for i in range(tot_natoms):
			line=ip.readline().split()
			coord.append(line[0:3])
			selective_tags.append(line[3:])
		try:
			coord=np.array(coord,dtype="d")
		except:
			raise IOError("POSCAR file is weird. In [%s] file, Maybe number of coordinations are not matched with total number of atoms"%(file_name))
		selective_tags=np.array(selective_tags,dtype=str)
	ip.close()
	return system_name, lattice, cell_vec, species, natoms, coord_type, coord, Selective, selective_tags

def write_POSCAR(output_file, system_name, lattice, cell_vec, species, natoms, coord_type, coord, Selective=False, selective_tags=[]):
	'''Write structure files of VASP program.
    [Input] : output_file(name of output file), lattice, cell_vec, species, natoms, coord_type, coord, Selective, selective_tags
	|-> system_name : name of system. @str
    |-> lattice : scale @float
    |-> cell_vec: [x1, y1, z1], [x2, y2, z2], [x3, y3, z3]] @np.array(dtype='d')
    |-> species: [atomt1, atom2, ...] @list(str)
    |-> natoms: [number_of_atom1, number_of_atom2, ...] @list(int)
	|-> coord_type: "DIRECT" / "CARTESIAN" @str
	|-> coord : coordination of each atoms @np.array(dtype='d')
    |-> Selective : True / False @bool
    |-> selective_tags : [[T/F, T/F, T/F], [T/F, T/F, T/F], ...] @np.array(dtype=str)
    [Output] : VASP format structure file.
    '''
	op=open(output_file,'w')
	print(system_name,file=op)
	print("   %16.14f"%(lattice),file=op)
	for i in range(3):
	    print("    %19.16f   %19.16f   %19.16f"%(cell_vec[i][0],cell_vec[i][1],cell_vec[i][2]),file=op)

	print("",end=" ",file=op)
	for i in range(len(species)):
	    print("%4s"%(species[i]),end="",file=op)
	print("",file=op)

	for i in range(len(natoms)):
	    print("%6d"%(natoms[i]),end="",file=op)
	print("",file=op)

	if Selective:
	    print("Selective dynamics",file=op)
	if coord_type=="CARTESIAN":
	    print("Cartesian",file=op)
	else:
	    print("Direct",file=op)

	if Selective:
	    for (x1,y1) in zip(coord,selective_tags):
	        for i in x1:
	            print("%20.16f"%(i),end="",file=op)
	        for p in y1:
	            print("%4s"%(p),end="",file=op)
	        print("",file=op)
	else:
	    for x1 in coord:
	        for i in x1:
	            print("%20.16f"%(i),end="",file=op)
	        print("",file=op)
	op.close()
	print("(+) Structure file [%s] was generated"%(output_file))

def dir2car(coord,cell_vec):
    '''Direct to Cartesian
    [Input] : coord, cell_vec
    |-> coord : coordination. @np.array(dtype="d")
    |-> cell_vec : cell vector. @np.array(dtype='d')
    [Output] : new coordination @np.array(dtype='d')
    '''
    return np.matmul(coord,cell_vec)

def car2dir(coord,cell_vec):
    '''Cartesian to Direct
    [Input] : coord, cell_vec
    |-> coord : coordination. @np.array(dtype='d')
    |-> cell_vec : cell vector. @np.array(dtype='d')
    [Output] : new coordination @np.array(dtype='d')
    '''
    inv_cell_vec=np.linalg.inv(cell_vec)
    return np.matmul(coord,inv_cell_vec)


def extract(object="OUTCAR",property="energy"):
	energy="cat OUTCAR | grep \"energy without entropy\" | tail -1 | awk '{printf \"%10.9f\", $5}'"
	fermi="grep fermi OUTCAR | tail -1 | awk '{printf\"%20.8f \",$3}'"
	volume="cat OUTCAR | grep volume | tail -1 | awk '{printf \"%20.8f\",$5}'"
	if property=="energy":
		# extract(property="energy")
		i=float(subprocess.check_output(energy,shell=True))
	elif property=="fermi":
		# extract(property="fermi")
		i=float(subprocess.check_output(fermi,shell=True))
	elif property=="volume":
		# extract(property="volume")
		i=float(subprocess.check_output(volume,shell=True))
	return i

def run_vasp(vasp_command):
	'''i=0 means exit 0, which is terminated normally. if not, it should be calculated again'''
	i=subprocess.call(vasp_command,shell=True)
	return i
#cal1_exit=run_vasp(z_opt)


def mkdir(dirname):
    directory = os.path.dirname(dirname+"/")
    if not os.path.exists(directory):
        os.makedirs(directory)


	#####

	  # def is_converge(self):
   #      if self.convergence_info is None or len(self.convergence_info) < 3:
   #          return False

   #      energies = [x['free_energy'] for x in self.convergence_info]
   #      if len(energies) > 2 and abs(max(energies[-3:]) - min(energies[-3:])) >= self.energy_tolerance:
   #          return False
   #      else:
   #          return True

def cartesian_strain(cell_vec,xstrain,ystrain):
	'''
	[Input] : cell vector, strain of x and y.
	|---> cell_vec : cell vector @np.array(dtype='d')
	|---> xstrain : applied strain along x directions. 1.00 means no applied strain @float
	|---> ystrain : applied strain along y directions. 1.00 means no applied strain @float
	[Output] : new cell_vector @np.array(dtype='d')
	'''
	cell_vec[0] *= xstrain
	cell_vec[1] *= ystrain
	return cell_vec


def angle_ab(a,b,rad=True,acute=True):
	'''
	Calculate internal angle between first vector (a1,a2,a3) and second vector(b1,b2,b3)
	[Input] a, b, rad, acute
	|---> a : first vector. @list/tuple/np.array
	|---> b : second vector. @list/tuple/np.array
	|---> rad : True means output's unit will be radian. False means output's unit will be degree @bool.
	|---> acute : acute literally means the acute angle. if True : output doens't exceed 90 deg, False doesn't care. @bool
	[Output] : Angles, unit is degree(when rad==False) or radian(when rad==True)
	'''
	from math import pi, cos, sin, acos, sqrt
	a1,a2,a3=a; b1,b2,b3=b
	ab=acos((a1*b1+a2*b2+a3*b3)/(sqrt(a1**2+a2**2+a3**2)*sqrt(b1**2+b2**2+b3**2)))
	if acute==True:
	    if ab > pi/2:
	          ab=pi-ab
	if rad==True:
	    return ab
	else:
	    return ab/pi*180


def diagonal_strain(cell_vec,theta,epsilon):
	'''
	[Input] : cell vector, theta, epsilon
	|---> theta : Angles between applied strain's axis and x axis. Units are degree. 0 means along x axis @float
	|---> epsilon : Amount of applied strain along strain axis. @float
	[Output] : new cell vector @np.array(dtype='d')
	'''
	# from https://github.com/materials-theory/sss_package/blob/master/surface_strain_radian_v0924.py
	from math import pi, cos, sin, acos, sqrt
	x1, x2, y1, y2 = cell_vec[0,0], cell_vec[0,1], cell_vec[1,0], cell_vec[1,1]
    # we will use radian
	theta_r=theta*pi/180

	# Defining strain vector (t1,t2,0)
	t1=cos(theta_r)*x1-sin(theta_r)*x2
	t2=sin(theta_r)*x1+cos(theta_r)*x2
	q_x = angle_ab((t1,t2,0),(x1,x2,0),rad=True,acute=True)
	q_y = angle_ab((t1,t2,0),(y1,y2,0),rad=True,acute=True)
	x1_t = (1+epsilon*cos(q_x))*x1
	x2_t = (1+epsilon*cos(q_x))*x2
	y1_t = (1+epsilon*cos(q_y))*y1
	y2_t = (1+epsilon*cos(q_y))*y2
	cell_vec = np.array([(x1_t,x2_t,0),(y1_t,y2_t,0),(0,0,cell_vec[2,2])],dtype='d')
	return cell_vec


In [11]:
wp=read_POSCAR("1.vasp")

In [15]:
sum(wp[4])

20

In [68]:
write_POSCAR("1-C.vasp",wp[0],wp[1],wp[2],wp[3],wp[4],"CARTESIAN",dir2car(wp[6],wp[2]))

(+) Structure file [1-C.vasp] was generated


In [116]:
wp2=read_POSCAR("1-C.vasp")

In [117]:
coord=wp2[6]

In [118]:
wp2[2][2][2]

62.0198426

In [127]:
nlayer=13
dlayer=0.5
flayer=3

In [119]:
#wp2 --> read_POSCAR한 거
if wp2[5]=="DIRECT":
    coord=dir2car(wp2[6],wp2[2])
else:
    coord=wp[6]
for i in range(len(coord[:,2])):
    if wp2[2][2][2]-coord[:,2][i]<dlayer:
        coord[:,2][i]-=wp2[2][2][2]

layers={}

# original layer 개수를 al이라는 변수로 받자


In [120]:
coord

array([[ 0.        ,  0.        , 60.31480686],
       [ 2.11986101,  1.2233336 ,  0.86540082],
       [ 2.11986096,  1.22333358, 11.25021215],
       [ 2.11986096,  1.22333358, 21.63502738],
       [ 2.11986096,  1.22333358, 32.01984262],
       [ 0.        ,  0.        ,  7.78860905],
       [ 0.        ,  0.        , 18.17342038],
       [ 0.        ,  0.        , 28.55823562],
       [ 4.23972154,  2.44666672,  4.32700491],
       [ 4.2397217 ,  2.44666679, 14.71181729],
       [ 4.2397217 ,  2.44666679, 25.09663246],
       [ 4.23972154,  2.44666672,  6.92320681],
       [ 4.2397217 ,  2.44666679, 17.3080182 ],
       [ 4.2397217 ,  2.44666679, 27.69283343],
       [ 2.11986101,  1.2233336 ,  3.46160365],
       [ 2.11986096,  1.22333358, 13.84641696],
       [ 2.11986096,  1.22333358, 24.23122643],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 10.38481182],
       [ 0.        ,  0.        , 20.76962328],
       [ 0.        ,  0.        , 31.154

In [85]:
temp_dict={}
temp_list=[]
tempn=1
temp_dict[1]=[]
for i in np.argsort(coord[:,2]):
    if tempn==1:
        temp_dict[tempn].append(i)
    else:
        if coord[:,2][temp_dict[tempn][0]]+dlayer
    if tempn!=1:
        temp_list.append(coord[:,2][np.argsort(coord[:,2])[tempn]]-coord[:,2][np.argsort(coord[:,2])[tempn-1]])
    tempn+=1

In [92]:
temp_dict[tempn]=list(1)

TypeError: 'int' object is not iterable

In [90]:
temp_dict[tempn]

1